In [1]:
!pip install transformers torch langchain sentencepiece

   ---------------------------------------- 0.0/10.4 MB ? eta -:--:--
   - -------------------------------------- 0.3/10.4 MB ? eta -:--:--
   -- ------------------------------------- 0.5/10.4 MB 1.6 MB/s eta 0:00:07
   ---- ----------------------------------- 1.0/10.4 MB 1.8 MB/s eta 0:00:06
   ----- ---------------------------------- 1.3/10.4 MB 2.0 MB/s eta 0:00:05
   -------- ------------------------------- 2.1/10.4 MB 2.2 MB/s eta 0:00:04
   ---------- ----------------------------- 2.6/10.4 MB 2.3 MB/s eta 0:00:04
   ------------- -------------------------- 3.4/10.4 MB 2.5 MB/s eta 0:00:03
   ---------------- ----------------------- 4.2/10.4 MB 2.7 MB/s eta 0:00:03
   --------------------- ------------------ 5.5/10.4 MB 3.1 MB/s eta 0:00:02
   ------------------------- -------------- 6.6/10.4 MB 3.3 MB/s eta 0:00:02
   ------------------------------- -------- 8.1/10.4 MB 3.7 MB/s eta 0:00:01
   ------------------------------------- -- 9.7/10.4 MB 4.0 MB/s eta 0:00:01
   ----------

In [1]:
with open("arimidex_full_parsed.md", "r", encoding="utf-8") as f:
    context = f.read()

print(f"仿單長度: {len(context)} 字")
print(context[:200]) # 檢查一下內容對不對

仿單長度: 5700 字
安美達錠1毫克
# Arimidex Tablets 1mg

| 非常常見         | 心血管：         | • 熱潮紅，通常為輕至中度      |
| ------------ | ------------ | ------------------ |
| 一般：          | • 無力，通常為輕至中度 | 肌肉骨骼和結締組織方面的異常：    |
| • 關節痛/關


In [2]:
from transformers import pipeline

# 載入一個已經會做中文問答的模型
# 這裡選用一個專精中文 extractive QA 的模型
qa_pipeline = pipeline(
    "question-answering",
    model="uer/roberta-base-chinese-extractive-qa",
    tokenizer="uer/roberta-base-chinese-extractive-qa"
)

print("模型載入完成！")

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertForQuestionAnswering LOAD REPORT from: uer/roberta-base-chinese-extractive-qa
Key                          | Status     |  | 
-----------------------------+------------+--+-
bert.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


模型載入完成！


## 改良的切分策略：依 Markdown 段落切塊

Extractive QA 的限制是：模型一次只能讀約 512 個字。直接把整份仿單丟進去會報錯。

**原本**的做法是每 400 字切一塊（固定長度），**缺點**是可能把一個完整段落切到兩半——例如「禁忌」的清單只有 200 字，卻和前一段的末尾拼在一起；「不良反應」的段落超過 400 字，被攔腰切斷後模型只看到一半的資訊。

**改良後**的做法：依 Markdown 的標題（`#`）來切段，讓每個片段剛好對應一個主題。同時把標題加在片段前面，讓模型知道這段文字的主題。

切出來的片段會長這樣：
```
【禁忌】
'Arimidex'禁用於：
- 停經前婦女
- 懷孕或授乳婦
...

【用法用量】
1. 成人(包括老人)：口服一天一錠 (1mg)。
...

【非預期的作用】
使用'Arimidex'之病患有較少之熱潮紅、陰道出血...
```

（備註：這就是為什麼 RAG 比較強，因為 RAG 還會再加上語意搜尋，Extractive QA 需要自己寫這些邏輯）

In [3]:
import re

# 改良切分策略：依 Markdown 標題切段，保留標題作為 context 前綴
# 優點：同一段落的資訊不會被切斷，「禁忌」、「用法用量」等各自獨立成塊
def chunk_by_sections(text, max_length=450):
    sections = []
    current_title = "藥品基本資訊"
    current_lines = []

    for line in text.split('\n'):
        if re.match(r'^#{1,3} ', line):
            # 儲存前一段
            content = '\n'.join(current_lines).strip()
            if content:
                prefix = f"【{current_title}】\n"
                full = prefix + content
                if len(full) <= max_length:
                    sections.append(full)
                else:
                    # 太長就再切，但每個子片段都保留標題前綴
                    step = max_length - len(prefix)
                    for i in range(0, len(content), step):
                        sections.append(prefix + content[i:i + step])
            current_title = re.sub(r'^#+\s*', '', line).rstrip('：').strip()
            current_lines = []
        else:
            current_lines.append(line)

    # 最後一段
    content = '\n'.join(current_lines).strip()
    if content:
        prefix = f"【{current_title}】\n"
        full = prefix + content
        if len(full) <= max_length:
            sections.append(full)
        else:
            step = max_length - len(prefix)
            for i in range(0, len(content), step):
                sections.append(prefix + content[i:i + step])

    return sections

chunks = chunk_by_sections(context)
print(f"總共切成 {len(chunks)} 個片段：")
for i, c in enumerate(chunks):
    first_line = c.split('\n')[0]
    print(f"  片段 {i:2d}: {first_line}（{len(c)} 字）")

總共切成 28 個片段：
  片段  0: 【藥品基本資訊】（16 字）
  片段  1: 【Arimidex Tablets 1mg】（450 字）
  片段  2: 【Arimidex Tablets 1mg】（46 字）
  片段  3: 【Anastrozole 膜衣錠】（81 字）
  片段  4: 【適應症】（139 字）
  片段  5: 【用法用量】（162 字）
  片段  6: 【禁忌】（194 字）
  片段  7: 【交互作用】（330 字）
  片段  8: 【過量】（271 字）
  片段  9: 【藥效動力學特性】（431 字）
  片段 10: 【妊娠與授乳】（29 字）
  片段 11: 【對開車或其他使用機械能力的影響】（97 字）
  片段 12: 【非預期的作用】（403 字）
  片段 13: 【活率之益處方面】（450 字）
  片段 14: 【活率之益處方面】（199 字）
  片段 15: 【輔助治療已使用過tamoxifen之早期乳癌】（250 字）
  片段 16: 【急性毒性】（130 字）
  片段 17: 【慢性毒性】（156 字）
  片段 18: 【突變性】（58 字）
  片段 19: 【生殖毒性】（208 字）
  片段 20: 【致癌性】（177 字）
  片段 21: 【脂質】（79 字）
  片段 22: 【兒童】（75 字）
  片段 23: 【男性女乳症臨床研究】（326 字）
  片段 24: 【馬科恩‑亞白特氏症候群(McCune Albright Syndrome, MAS)臨床研究】（352 字）
  片段 25: 【整體評估】（32 字）
  片段 26: 【藥物動力學特性】（250 字）
  片段 27: 【製造廠】（242 字）


In [4]:
# 定義問答函數（同時回傳答案來源的段落，方便 debug）
def ask_arimidex(question):
    best_answer = {"score": 0, "answer": "找不到答案", "section": ""}

    for i, chunk in enumerate(chunks):
        try:
            result = qa_pipeline(question=question, context=chunk)
            if result['score'] > best_answer['score']:
                best_answer = result
                best_answer['chunk_id'] = i
                best_answer['section'] = chunk.split('\n')[0]  # 記錄答案來自哪個段落
        except:
            continue

    return best_answer

In [5]:
question = "這藥的適應症是什麼？"
result = ask_arimidex(question)

print(f"問題: {question}")
print(f"答案: {result['answer']}")
print(f"來源段落: {result.get('section', '未知')}")
print(f"信心分數: {result['score']:.4f}")

問題: 這藥的適應症是什麼？
答案: 治療停經後婦女晚期乳癌
來源段落: 【適應症】
信心分數: 0.4547


In [6]:
question = "不良反應包含哪些？"
result = ask_arimidex(question)

print(f"問題: {question}")
print(f"答案: {result['answer']}")
print(f"來源段落: {result.get('section', '未知')}")
print(f"信心分數: {result['score']:.4f}")

問題: 不良反應包含哪些？
答案: 關節炎、關節退化及關節痛
來源段落: 【活率之益處方面】
信心分數: 0.0084


In [7]:
question = "什麼人不能吃？"
result = ask_arimidex(question)

print(f"問題: {question}")
print(f"答案: {result['answer']}")
print(f"來源段落: {result.get('section', '未知')}")
print(f"信心分數: {result['score']:.4f}")

問題: 什麼人不能吃？
答案: 狗
來源段落: 【急性毒性】
信心分數: 0.0006


In [8]:
question = "怎麼服用？"
result = ask_arimidex(question)

print(f"問題: {question}")
print(f"答案: {result['answer']}")
print(f"來源段落: {result.get('section', '未知')}")
print(f"信心分數: {result['score']:.4f}")

問題: 怎麼服用？
答案: 口服
來源段落: 【用法用量】
信心分數: 0.0964


In [9]:
question = "一次吃幾顆？"
result = ask_arimidex(question)

print(f"問題: {question}")
print(f"答案: {result['answer']}")
print(f"來源段落: {result.get('section', '未知')}")
print(f"信心分數: {result['score']:.4f}")

問題: 一次吃幾顆？
答案: 2
來源段落: 【適應症】
信心分數: 0.0001
